In [2]:
import random
import numpy as np
import pandas as pd
import enum
import time
from termcolor import colored

In [13]:
xl = pd.read_excel("lab3.xlsx", sheet_name="Лист1")
xl

,СОСТОЯНИЯ под каждые этажи,ПРОВЕРКИ,СЛЕДУЮЩЕЕ СОСТОЯНИЕ,ВЗЯТЬ СЛЕДУЮЩЕЕ,ИСКЛЮЧЕНИЕ,ВЫХОД,Вывод (callback)
0,1) стоит стоит,empty,-,-,0,1,-
1,2) выбран стоит,1 лифт ближе к старту,4,-,0,-,-
2,3) стоит выбран,2 лфит ближе к старту,12,-,1,-,-
3,4) вверх стоит,1 лифт ниже старта,4 + выше этаж,-,0,-,Лифт 1 на + этаж
4,5) вниз стоит,1 лифт выше старта,5 + ниже этаж,-,0,-,Лифт 1 на - этаж
5,6) открыть стоит,1 лифт на старте,7,-,1,-,Лифт 1: открыт
6,7) закртыть стоит,1 лифт на старте,8,-,1,-,Лифт 1: запустил + закрыл
7,-,-,-,-,-,-,-
8,8) вверх2 стоит,1 лифт ниже финиша,8 + выше этаж,-,0,-,Лифт 1 на + этаж
9,9) вниз2 стоит,1 лифт выше финиша,9 + ниже этаж,-,0,-,Лифт 1 на - этаж


In [3]:
class LiftState(enum.Enum):
    STAY = 0
    SELECTED = 1
    UP = 2
    DOWN = 3
    OPEN = 4
    CLOSE = 5

    UP2 = 6
    DOWN2 = 7
    OPEN2 = 8
    CLOSE2 = 9

In [4]:
class TwoLifts:
    travels = None

    def __init__(self, floors: int, start_state: tuple, print_movments: bool = False):
        self.floors = floors

        self.state = (*start_state, LiftState.STAY, LiftState.STAY)
        # self.state = (2, 3, LiftState.STAY, LiftState.STAY)
        self.states_dca = {}
        self.move_count = 0
        self.all_move_count = 0
        self.print_movments = print_movments
        self.iter_command = None

    def plus_move_count(self):
        self.move_count += 1

    def plus_all_move_count(self):
        self.all_move_count += self.move_count
        self.move_count = 0

    def reset(self):
        self.state = (1, 1, LiftState.STAY, LiftState.STAY)
        self.move_count = 0
        self.all_move_count = 0
        self.iter_command = None

    def run(self, commands: list):
        commands_copy = commands.copy()
        commands_copy.append(None)
        self.iter_command = commands_copy.pop(0)
        print(colored("Следующий элемент: " + str(self.iter_command), "light_green"))
        while True:
            # print("stete: " + str(self.state))
            dca_item = self.states_dca[self.state]
            if dca_item["if"](self.iter_command):
                dca_item["function"]()
                if dca_item["iter_next"]:
                    self.iter_command = commands_copy.pop(0)
                    print(colored("Следующий элемент: " + str(self.iter_command), "light_green"))
                if dca_item["can_exit"]:
                    break
                self.state = dca_item["next_state"]
            else:
                if dca_item["exception_next_state"] != None:
                    self.state = dca_item["exception_next_state"]
                else:
                    raise Exception("Ошибка состояния")
        print(f"Всего операций: {self.all_move_count}")



    def generateDCA(self):
        for i in range(1, self.floors + 1):
            for j in range(1, self.floors + 1):
                #1
                self.states_dca[i, j, LiftState.STAY, LiftState.STAY] = {
                    "if": lambda iter_value: iter_value == None,
                    "next_state": None,
                    "iter_next": False,
                    "exception_next_state": (i, j, LiftState.SELECTED, LiftState.STAY),
                    "can_exit": True,
                    "function": self.plus_move_count
                }
                #2
                self.states_dca[i, j, LiftState.SELECTED, LiftState.STAY] = {
                    "if": lambda iter_value: abs(self.state[0] - iter_value[0]) <= abs(self.state[1] - iter_value[0]),
                    "next_state": (i, j, LiftState.UP, LiftState.STAY) if i != self.floors else (i, j, LiftState.DOWN, LiftState.STAY),
                    "iter_next": False,
                    "exception_next_state": (i, j, LiftState.STAY, LiftState.SELECTED),
                    "can_exit": False,
                    "function": self.plus_move_count
                }
                #3
                self.states_dca[i, j, LiftState.STAY, LiftState.SELECTED] = {
                    "if": lambda iter_value: True, #lambda iter_value: abs(self.state[0] - iter_value[0]) > abs(self.state[1] - iter_value[1]),
                    "next_state": (i, j, LiftState.STAY, LiftState.UP) if j != self.floors else (i, j, LiftState.STAY, LiftState.DOWN),
                    "iter_next": False,
                    "exception_next_state": None,
                    "can_exit": False,
                    "function": self.plus_move_count
                }
                #4
                if i != self.floors:
                    self.states_dca[i, j, LiftState.UP, LiftState.STAY] = {
                        "if": lambda iter_value: self.state[0] < iter_value[0],
                        "next_state": (i + 1, j, LiftState.UP, LiftState.STAY) if i + 1 != self.floors else (i + 1, j, LiftState.OPEN, LiftState.STAY),
                        "iter_next": False,
                        "exception_next_state": (i, j, LiftState.DOWN, LiftState.STAY) if i != 1 else (i, j, LiftState.OPEN, LiftState.STAY),
                        "can_exit": False,
                        "function": lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0] + 1}, {self.state[1]}]") if self.print_movments else None)
                    }
                #5
                if i != 1:
                    self.states_dca[i, j, LiftState.DOWN, LiftState.STAY] = {
                        "if": lambda iter_value: self.state[0] > iter_value[0],
                        "next_state": (i - 1, j, LiftState.DOWN, LiftState.STAY) if i - 1 != 1 else (i - 1, j, LiftState.OPEN, LiftState.STAY),
                        "iter_next": False,
                        "exception_next_state": (i, j, LiftState.OPEN, LiftState.STAY),
                        "can_exit": False,
                        "function": lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0] - 1}, {self.state[1]}]") if self.print_movments else None)
                    }
                #6
                self.states_dca[i, j, LiftState.OPEN, LiftState.STAY] = {
                    "if": lambda iter_value: self.state[0] == iter_value[0],
                    "next_state": (i, j, LiftState.CLOSE, LiftState.STAY),
                    "iter_next": False,
                    "exception_next_state": None,
                    "can_exit": False,
                    "function": lambda: (self.plus_move_count(), print("Лифт 1: открыт") if self.print_movments else None)
                }
                #7
                self.states_dca[i, j, LiftState.CLOSE, LiftState.STAY] = {
                    "if": lambda iter_value: True,
                    "next_state": (i, j, LiftState.UP2, LiftState.STAY) if i != self.floors else (i, j, LiftState.DOWN2, LiftState.STAY),
                    "iter_next": False,
                    "exception_next_state": None,
                    "can_exit": False,
                    "function": lambda: (self.plus_move_count(), print("Лифт 1: закрыт") if self.print_movments else None)
                }
                #8
                if i != self.floors:
                    self.states_dca[i, j, LiftState.UP2, LiftState.STAY] = {
                        "if": lambda iter_value: self.state[0] < iter_value[1],
                        "next_state": (i + 1, j, LiftState.UP2, LiftState.STAY) if i + 1 != self.floors else (i + 1, j, LiftState.OPEN2, LiftState.STAY),
                        "iter_next": False,
                        "exception_next_state": (i, j, LiftState.DOWN2, LiftState.STAY) if i != 1 else (i, j, LiftState.OPEN2, LiftState.STAY),
                        "can_exit": False,
                        "function": lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0] + 1}, {self.state[1]}]") if self.print_movments else None)
                    }
                #9
                if i != 1:
                    self.states_dca[i, j, LiftState.DOWN2, LiftState.STAY] = {
                        "if": lambda iter_value: self.state[0] > iter_value[1],
                        "next_state": (i - 1, j, LiftState.DOWN2, LiftState.STAY) if i - 1 != 1 else (i - 1, j, LiftState.OPEN2, LiftState.STAY),
                        "iter_next": False,
                        "exception_next_state": (i, j, LiftState.OPEN2, LiftState.STAY),
                        "can_exit": False,
                        "function": lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0] - 1}, {self.state[1]}]") if self.print_movments else None)
                    }
                #10
                self.states_dca[i, j, LiftState.OPEN2, LiftState.STAY] = {
                    "if": lambda iter_value: self.state[0] == iter_value[1],
                    "next_state": (i, j, LiftState.CLOSE2, LiftState.STAY),
                    "iter_next": False,
                    "exception_next_state": None,
                    "can_exit": False,
                    "function": lambda: (self.plus_move_count(), print("Лифт 1: открыт") if self.print_movments else None)
                }
                #11
                self.states_dca[i, j, LiftState.CLOSE2, LiftState.STAY] = {
                    "if": lambda iter_value: True,
                    "next_state": (i, j, LiftState.STAY, LiftState.STAY),
                    "iter_next": True,
                    "exception_next_state": None,
                    "can_exit": False,
                    "function": lambda: (self.plus_move_count(), print("Лифт 1: закрыт") if self.print_movments else None, 
                                         print(colored(f"Для вызова ({self.iter_command}) было затрачено {self.move_count} операций.", "light_green")), self.plus_all_move_count())
                }
                #12
                if j != self.floors:
                    self.states_dca[i, j, LiftState.STAY, LiftState.UP] = {
                        "if": lambda iter_value: self.state[1] < iter_value[0],
                        "next_state": (i, j + 1, LiftState.STAY, LiftState.UP) if j + 1 != self.floors else (i, j + 1, LiftState.STAY, LiftState.OPEN),
                        "iter_next": False,
                        "exception_next_state": (i, j, LiftState.STAY, LiftState.DOWN) if j != 1 else (i, j, LiftState.STAY, LiftState.OPEN),
                        "can_exit": False,
                        "function": lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0]}, {self.state[1] + 1}]") if self.print_movments else None)
                    }
                #13
                if j != 1:
                    self.states_dca[i, j, LiftState.STAY, LiftState.DOWN] = {
                        "if": lambda iter_value: self.state[1] > iter_value[0],
                        "next_state": (i, j - 1, LiftState.STAY, LiftState.DOWN) if j - 1 != 1 else (i, j - 1, LiftState.STAY, LiftState.OPEN),
                        "iter_next": False,
                        "exception_next_state": (i, j, LiftState.STAY, LiftState.OPEN),
                        "can_exit": False,
                        "function": lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0]}, {self.state[1] - 1}]") if self.print_movments else None)
                    }
                #14
                self.states_dca[i, j, LiftState.STAY, LiftState.OPEN] = {
                    "if": lambda iter_value: self.state[1] == iter_value[0],
                    "next_state": (i, j, LiftState.STAY, LiftState.CLOSE),
                    "iter_next": False,
                    "exception_next_state": None,
                    "can_exit": False,
                    "function": lambda: (self.plus_move_count(), print("Лифт 2: открыт") if self.print_movments else None)
                }
                #15
                self.states_dca[i, j, LiftState.STAY, LiftState.CLOSE] = {
                    "if": lambda iter_value: True,
                    "next_state": (i, j, LiftState.STAY, LiftState.UP2) if j != self.floors else (i, j, LiftState.STAY, LiftState.DOWN2),
                    "iter_next": False,
                    "exception_next_state": None,
                    "can_exit": False,
                    "function": lambda: (self.plus_move_count(), print("Лифт 2: закрыт") if self.print_movments else None)
                }
                #16
                if j != self.floors:
                    self.states_dca[i, j, LiftState.STAY, LiftState.UP2] = {
                        "if": lambda iter_value: self.state[1] < iter_value[1],
                        "next_state": (i, j + 1, LiftState.STAY, LiftState.UP2) if j + 1 != self.floors else (i, j + 1, LiftState.STAY, LiftState.OPEN2),
                        "iter_next": False,
                        "exception_next_state": (i, j, LiftState.STAY, LiftState.DOWN2) if j != 1 else (i, j, LiftState.STAY, LiftState.OPEN2),
                        "can_exit": False,
                        "function": lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0]}, {self.state[1] + 1}]") if self.print_movments else None)
                    }
                #17
                if j != 1:
                    self.states_dca[i, j, LiftState.STAY, LiftState.DOWN2] = {
                        "if": lambda iter_value: self.state[1] > iter_value[1],
                        "next_state": (i, j - 1, LiftState.STAY, LiftState.DOWN2) if j - 1 != 1 else (i, j - 1, LiftState.STAY, LiftState.OPEN2),
                        "iter_next": False,
                        "exception_next_state": (i, j, LiftState.STAY, LiftState.OPEN2),
                        "can_exit": False,
                        "function": lambda: (self.plus_move_count(), print(f"Лифты: [{self.state[0]}, {self.state[1]}]->[{self.state[0]}, {self.state[1] - 1}]") if self.print_movments else None)
                    }
                #18
                self.states_dca[i, j, LiftState.STAY, LiftState.OPEN2] = {
                    "if": lambda iter_value: self.state[1] == iter_value[1],
                    "next_state": (i, j, LiftState.STAY, LiftState.CLOSE2),
                    "iter_next": False,
                    "exception_next_state": None,
                    "can_exit": False,
                    "function": lambda: (self.plus_move_count(), print("Лифт 2: открыт") if self.print_movments else None)
                }
                #19
                self.states_dca[i, j, LiftState.STAY, LiftState.CLOSE2] = {
                    "if": lambda iter_value: True,
                    "next_state": (i, j, LiftState.STAY, LiftState.STAY),
                    "iter_next": True,
                    "exception_next_state": None,
                    "can_exit": False,
                    "function": lambda: (self.plus_move_count(), print("Лифт 2: закрыт") if self.print_movments else None, 
                                         print(colored(f"Для вызова ({self.iter_command}) было затрачено {self.move_count} операций.", "light_green")), self.plus_all_move_count())
                }

                


    
    

In [5]:
floors = list(range(1, random.randint(5, 21)))
print(floors[-1])
actions_count = random.randint(5, 10)
actions = [np.random.choice(floors, 2).tolist() for i in range(actions_count)]
print(actions)

15
[[3, 1], [15, 4], [3, 4], [1, 15], [12, 3], [3, 13], [10, 9]]


In [6]:
t = TwoLifts(floors=floors[-1], start_state=(1, 1), print_movments=True)
t.generateDCA()
t.run(actions)

Следующий элемент: [3, 1]
Лифты: [1, 1]->[2, 1]
Лифты: [2, 1]->[3, 1]
Лифт 1: открыт
Лифт 1: закрыт
Лифты: [3, 1]->[2, 1]
Лифты: [2, 1]->[1, 1]
Лифт 1: открыт
Лифт 1: закрыт
Для вызова ([3, 1]) было затрачено 9 операций.
Следующий элемент: [15, 4]
Лифты: [1, 1]->[2, 1]
Лифты: [2, 1]->[3, 1]
Лифты: [3, 1]->[4, 1]
Лифты: [4, 1]->[5, 1]
Лифты: [5, 1]->[6, 1]
Лифты: [6, 1]->[7, 1]
Лифты: [7, 1]->[8, 1]
Лифты: [8, 1]->[9, 1]
Лифты: [9, 1]->[10, 1]
Лифты: [10, 1]->[11, 1]
Лифты: [11, 1]->[12, 1]
Лифты: [12, 1]->[13, 1]
Лифты: [13, 1]->[14, 1]
Лифты: [14, 1]->[15, 1]
Лифт 1: открыт
Лифт 1: закрыт
Лифты: [15, 1]->[14, 1]
Лифты: [14, 1]->[13, 1]
Лифты: [13, 1]->[12, 1]
Лифты: [12, 1]->[11, 1]
Лифты: [11, 1]->[10, 1]
Лифты: [10, 1]->[9, 1]
Лифты: [9, 1]->[8, 1]
Лифты: [8, 1]->[7, 1]
Лифты: [7, 1]->[6, 1]
Лифты: [6, 1]->[5, 1]
Лифты: [5, 1]->[4, 1]
Лифт 1: открыт
Лифт 1: закрыт
Для вызова ([15, 4]) было затрачено 30 операций.
Следующий элемент: [3, 4]
Лифты: [4, 1]->[3, 1]
Лифт 1: открыт
Лифт 1: 

In [ ]:
# floors = list(range(1, 11))
# t = TwoLifts(floors=floors[-1], print_movments=False)
# t.generateDCA()
# for i in range(10000):
#     actions_count = random.randint(5, 10)
#     actions = [np.random.choice(floors, 2).tolist() for i in range(actions_count)]
#     t.run(actions)
#     t.reset()
